In [361]:
import pandas as pd

In [363]:
path="traffic_stops - traffic_stops_with_vehicle_number (1).csv"
df=pd.read_csv(path)
df.head()

C:\Users\ikpra\AppData\Local\Temp\ipykernel_22072\133511210.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [365]:
df.isnull().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [367]:
df.dropna(subset=['search_type'],inplace=True)
df.head(10)

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
5,2020-01-01,0:05:00,Canada,F,44,73,Other,Drunk Driving,Other,True,Vehicle Search,Ticket,False,16-30 Min,False,MH89ZF7676
6,2020-01-01,0:06:00,USA,F,80,70,White,Signal Violation,Speeding,False,Vehicle Search,Ticket,False,30+ Min,False,KA40AD3153
8,2020-01-01,0:08:00,USA,F,80,30,Hispanic,Other,Seatbelt,True,Frisk,Arrest,False,0-15 Min,True,DL50PO5101
11,2020-01-01,0:11:00,Canada,F,58,53,Hispanic,Signal Violation,Signal,False,Vehicle Search,Arrest,True,30+ Min,True,KA61JB1004
12,2020-01-01,0:12:00,India,M,51,46,Asian,Drunk Driving,Signal,False,Vehicle Search,Ticket,False,16-30 Min,True,WB70IV9884


In [369]:
import mysql.connector
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)
mycursor=mydb.cursor()
print("your SQL is connected")

your SQL is connected


In [371]:
mycursor.execute("CREATE DATABASE police_log")

In [373]:
mycursor.execute("USE police_log")
mycursor.execute("""
CREATE TABLE police_log(
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age_raw VARCHAR(20),
    driver_age INT(2),
    driver_race VARCHAR(50),
    violation_raw TEXT,
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(50)
)
""")
mydb.commit()

In [375]:
from tabulate import tabulate

In [377]:
mycursor.execute("SELECT * FROM police_log")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='psql'))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# MEDIUM LEVEL

# Query 1 Total Number of Police Stops

In [381]:
mycursor.execute("SELECT COUNT(*) AS total_police_stops FROM police_log")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+----------------------+
|   total_police_stops |
+======================+
|                65539 |
+----------------------+


# Query 2 Count of Stops by Violation Type

In [384]:
mycursor.execute("SELECT violation,COUNT(*) AS stops_count FROM police_log GROUP BY violation")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-------------+---------------+
| violation   |   stops_count |
+=============+===============+
| DUI         |         13075 |
+-------------+---------------+
| Other       |         13194 |
+-------------+---------------+
| Seatbelt    |         13007 |
+-------------+---------------+
| Signal      |         13112 |
+-------------+---------------+
| Speeding    |         13150 |
+-------------+---------------+
| violation   |             1 |
+-------------+---------------+


# Query 3 Number of Arrests vs. Warnings

In [388]:
mycursor.execute("SELECT stop_outcome,COUNT(*) AS outcome_count FROM police_log WHERE stop_outcome IN('Arrest','Warning') GROUP BY stop_outcome")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+----------------+-----------------+
| stop_outcome   |   outcome_count |
+================+=================+
| Arrest         |           21734 |
+----------------+-----------------+
| Warning        |           21966 |
+----------------+-----------------+


# Query 4 Average Age of Drivers Stopped


In [391]:
mycursor.execute("SELECT AVG(driver_age) AS avg_driver_age FROM police_log WHERE driver_age IS NOt NULL")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+------------------+
|   avg_driver_age |
+==================+
|          49.1115 |
+------------------+


# Query 5 Top 5 Most Frequent Search Types


In [394]:
mycursor.execute("SELECT search_type,COUNT(*) AS search_count FROM police_log WHERE search_type IS NOt NULL GROUP BY search_type LIMIT 5")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+----------------+----------------+
| search_type    |   search_count |
+================+================+
| Frisk          |          21971 |
+----------------+----------------+
| None           |          21720 |
+----------------+----------------+
| search_type    |              1 |
+----------------+----------------+
| Vehicle Search |          21847 |
+----------------+----------------+


# Query 6 Count of Stops by Gender

In [397]:
mycursor.execute("SELECT driver_gender,COUNT(*) AS stop_count FROM police_log GROUP BY driver_gender")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-----------------+--------------+
| driver_gender   |   stop_count |
+=================+==============+
| driver_gen      |            1 |
+-----------------+--------------+
| F               |        32881 |
+-----------------+--------------+
| M               |        32657 |
+-----------------+--------------+


# Query 7 Most Common Violation for Arrests

In [400]:
mycursor.execute("SELECT violation,COUNT(*) AS arrest_count FROM police_log WHERE is_arrested=TRUE GROUP BY violation LIMIT 1")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-------------+----------------+
| violation   | arrest_count   |
+=============+================+
+-------------+----------------+


# Query 8 Average Stop Duration for Each Violation

In [403]:
mycursor.execute("SELECT violation,stop_duration,COUNT(*) AS count FROM police_log GROUP BY violation,stop_duration")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-------------+-----------------+---------+
| violation   | stop_duration   |   count |
+=============+=================+=========+
| DUI         | 0-15 Min        |    4414 |
+-------------+-----------------+---------+
| DUI         | 16-30 Min       |    4331 |
+-------------+-----------------+---------+
| DUI         | 30+ Min         |    4330 |
+-------------+-----------------+---------+
| Other       | 0-15 Min        |    4265 |
+-------------+-----------------+---------+
| Other       | 16-30 Min       |    4296 |
+-------------+-----------------+---------+
| Other       | 30+ Min         |    4633 |
+-------------+-----------------+---------+
| Seatbelt    | 0-15 Min        |    4343 |
+-------------+-----------------+---------+
| Seatbelt    | 16-30 Min       |    4319 |
+-------------+-----------------+---------+
| Seatbelt    | 30+ Min         |    4345 |
+-------------+-----------------+---------+
| Signal      | 0-15 Min        |    4434 |
+-------------+-----------------

# Query 9 Number of Drug-Related Stops by Year

In [406]:
mycursor.execute("SELECT EXTRACT(YEAR FROM stop_date) AS year,COUNT(*) AS drugs_related_count FROM police_log WHERE drugs_related_stop=TRUE GROUP BY year")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+--------+-----------------------+
| year   | drugs_related_count   |
+========+=======================+
+--------+-----------------------+


# Query 10 Drivers with the Highest Number of Stops


In [409]:
mycursor.execute("SELECT vehicle_number,COUNT(*) AS count_stop FROM police_log GROUP BY vehicle_number ORDER BY count_stop DESC LIMIT 5")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+------------------+--------------+
| vehicle_number   |   count_stop |
+==================+==============+
| RJ46MS5966       |            1 |
+------------------+--------------+
| MH90YI4159       |            1 |
+------------------+--------------+
| RJ13JT2037       |            1 |
+------------------+--------------+
| UP76OH8225       |            1 |
+------------------+--------------+
| KA40IW6010       |            1 |
+------------------+--------------+


# Query 12 Number of Stops Conducted at Night (Between 10 PM - 5 AM)


In [412]:
mycursor.execute("SELECT COUNT(*) AS night_stops FROM police_log WHERE (CAST(SUBSTR(stop_time, 1, 2) AS INTEGER) >= 22 OR CAST(SUBSTR(stop_time, 1, 2) AS INTEGER) < 5)")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+---------------+
|   night_stops |
+===============+
|         19201 |
+---------------+


# Query 12 Number of Searches Conducted by Violation Type

In [415]:
mycursor.execute("SELECT violation,COUNT(*) AS searches_conducted FROM police_log WHERE search_conducted = TRUE GROUP BY violation")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-------------+----------------------+
| violation   | searches_conducted   |
+=============+======================+
+-------------+----------------------+


# Query 13 Arrest Rate by Driver Gender

In [418]:
mycursor.execute("SELECT driver_gender,COUNT(CASE WHEN is_arrested = 'True' THEN 1 END) AS arrests_count,COUNT(*) AS total_stops,(COUNT(CASE WHEN is_arrested = 'True' THEN 1 END) * 100.0 / COUNT(*)) AS arrest_rate_percentage FROM police_log GROUP BY driver_gender")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+-----------------+-----------------+---------------+--------------------------+
| driver_gender   |   arrests_count |   total_stops |   arrest_rate_percentage |
+=================+=================+===============+==========================+
| driver_gen      |               1 |             1 |                      100 |
+-----------------+-----------------+---------------+--------------------------+
| F               |           32881 |         32881 |                      100 |
+-----------------+-----------------+---------------+--------------------------+
| M               |           32657 |         32657 |                      100 |
+-----------------+-----------------+---------------+--------------------------+


# Query 14 Violation Trends Over Time (Monthly Count of Violations)

In [421]:
mycursor.execute("SELECT EXTRACT(YEAR FROM stop_date) AS year,EXTRACT(MONTH FROM stop_date) AS month,violation,COUNT(*) AS violation_count FROM police_log GROUP BY year,month,violation")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+--------+---------+-------------+-------------------+
|   year |   month | violation   |   violation_count |
+========+=========+=============+===================+
|      0 |       0 | violation   |                 1 |
+--------+---------+-------------+-------------------+
|   2020 |       1 | DUI         |              8906 |
+--------+---------+-------------+-------------------+
|   2020 |       1 | Other       |              8970 |
+--------+---------+-------------+-------------------+
|   2020 |       1 | Seatbelt    |              8899 |
+--------+---------+-------------+-------------------+
|   2020 |       1 | Signal      |              8923 |
+--------+---------+-------------+-------------------+
|   2020 |       1 | Speeding    |              8942 |
+--------+---------+-------------+-------------------+
|   2020 |       2 | DUI         |              4169 |
+--------+---------+-------------+-------------------+
|   2020 |       2 | Other       |              4224 |
+--------+

# Query 15 Most Common Stop Outcomes for Drug-Related Stops

In [424]:
mycursor.execute("SELECT stop_outcome,COUNT(*) AS outcome_count FROM police_log WHERE drugs_related_stop='True' GROUP BY stop_outcome")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='grid'))

+----------------+-----------------+
| stop_outcome   |   outcome_count |
+================+=================+
| Arrest         |           21734 |
+----------------+-----------------+
| stop_outcome   |               1 |
+----------------+-----------------+
| Ticket         |           21838 |
+----------------+-----------------+
| Warning        |           21966 |
+----------------+-----------------+


# COMPLEX LEVEL

# Query 1 Yearly Breakdown of Stops and Arrests by Country

In [428]:
mycursor.execute("""
WITH stops_data AS (
    SELECT 
        country_name,
        EXTRACT(YEAR FROM stop_date) AS stop_year,
        COUNT(*) AS total_stops,
        SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests 
    FROM police_log 
    GROUP BY country_name, EXTRACT(YEAR FROM stop_date)
)
SELECT * FROM stops_data
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+-------------+---------------+-----------------+
| country_name   |   stop_year |   total_stops |   total_arrests |
+================+=============+===============+=================+
| Canada         |        2020 |         21908 |               0 |
+----------------+-------------+---------------+-----------------+
| country_name   |           0 |             1 |               0 |
+----------------+-------------+---------------+-----------------+
| India          |        2020 |         21998 |               0 |
+----------------+-------------+---------------+-----------------+
| USA            |        2020 |         21632 |               0 |
+----------------+-------------+---------------+-----------------+


# Query 2 Driver Violation Trends Based on Age and Race

In [431]:
mycursor.execute("""
SELECT
    CASE 
        WHEN driver_age BETWEEN 16 AND 25 THEN '16-25'
        WHEN driver_age BETWEEN 26 AND 35 THEN '26-35'
        WHEN driver_age BETWEEN 36 AND 50 THEN '36-50'
        WHEN driver_age > 50 THEN '51+'
        ELSE 'Unknown'
    END AS age_group,
    driver_race,
    violation,
    COUNT(*) AS total_violations
FROM
    police_log

WHERE
    driver_age IS NOT NULL
    AND driver_race IS NOT NULL
    AND violation IS NOT NULL
GROUP BY
    age_group, driver_race, violation
ORDER BY
    age_group, driver_race, total_violations DESC
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+-------------+---------------+-------------+--------------------+
| age_group   | driver_race   | violation   |   total_violations |
+=============+===============+=============+====================+
| 16-25       | Asian         | Speeding    |                348 |
+-------------+---------------+-------------+--------------------+
| 16-25       | Asian         | Seatbelt    |                330 |
+-------------+---------------+-------------+--------------------+
| 16-25       | Asian         | Other       |                310 |
+-------------+---------------+-------------+--------------------+
| 16-25       | Asian         | DUI         |                302 |
+-------------+---------------+-------------+--------------------+
| 16-25       | Asian         | Signal      |                297 |
+-------------+---------------+-------------+--------------------+
| 16-25       | Black         | Speeding    |                365 |
+-------------+---------------+-------------+-----------------

# Query 3 Time Period Analysis of Stops

In [434]:
mycursor.execute("""
WITH time_periods AS (
    SELECT
        stop_date,
        stop_time,
        CASE 
            WHEN EXTRACT(HOUR FROM stop_time) BETWEEN 5 AND 11 THEN 'Morning'
            WHEN EXTRACT(HOUR FROM stop_time) BETWEEN 12 AND 16 THEN 'Afternoon'
            WHEN EXTRACT(HOUR FROM stop_time) BETWEEN 17 AND 20 THEN 'Evening'
            WHEN EXTRACT(HOUR FROM stop_time) BETWEEN 21 AND 23 THEN 'Night'
            ELSE 'Late Night'
        END AS time_period
    FROM
        police_log
)

-- Main query: count stops by date and time period
SELECT
    DATE(stop_date) AS stop_day,
    time_period,
    COUNT(*) AS total_stops
FROM
    time_periods
GROUP BY
    DATE(stop_date), time_period
ORDER BY
    stop_day, time_period

""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+------------+---------------+---------------+
| stop_day   | time_period   |   total_stops |
+============+===============+===============+
|            | Late Night    |             1 |
+------------+---------------+---------------+
| 2020-01-01 | Afternoon     |           300 |
+------------+---------------+---------------+
| 2020-01-01 | Evening       |           240 |
+------------+---------------+---------------+
| 2020-01-01 | Late Night    |           300 |
+------------+---------------+---------------+
| 2020-01-01 | Morning       |           420 |
+------------+---------------+---------------+
| 2020-01-01 | Night         |           180 |
+------------+---------------+---------------+
| 2020-01-02 | Afternoon     |           300 |
+------------+---------------+---------------+
| 2020-01-02 | Evening       |           240 |
+------------+---------------+---------------+
| 2020-01-02 | Late Night    |           300 |
+------------+---------------+---------------+
| 2020-01-02 

# Query 4 Violations with High Search and Arrest Rates

In [437]:
mycursor.execute("""WITH violation_stats AS (
    SELECT
        violation,
        COUNT(*) AS total_stops,
        SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) AS total_searches,
        SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests
    FROM
        police_log
    WHERE
        violation IS NOT NULL
    GROUP BY
        violation
)

SELECT
    violation,
    total_stops,
    total_searches,
    total_arrests,
    ROUND((total_searches * 100.0) / total_stops, 2) AS search_rate_percent,
    ROUND((total_arrests * 100.0) / total_stops, 2) AS arrest_rate_percent,
    RANK() OVER (ORDER BY (total_searches * 1.0 / total_stops) DESC) AS search_rank,
    RANK() OVER (ORDER BY (total_arrests * 1.0 / total_stops) DESC) AS arrest_rank
FROM
    violation_stats
ORDER BY
    search_rate_percent DESC, arrest_rate_percent DESC



""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+-------------+---------------+------------------+-----------------+-----------------------+-----------------------+---------------+---------------+
| violation   |   total_stops |   total_searches |   total_arrests |   search_rate_percent |   arrest_rate_percent |   search_rank |   arrest_rank |
+=============+===============+==================+=================+=======================+=======================+===============+===============+
| violation   |             1 |                0 |               0 |                     0 |                     0 |             1 |             1 |
+-------------+---------------+------------------+-----------------+-----------------------+-----------------------+---------------+---------------+
| Speeding    |         13150 |                0 |               0 |                     0 |                     0 |             1 |             1 |
+-------------+---------------+------------------+-----------------+-----------------------+--------------

# Query 5 Driver Demographics by Country

In [440]:
mycursor.execute("""SELECT
    country_name,
    driver_gender,
    driver_race,
    ROUND(AVG(driver_age), 1) AS average_age,
    COUNT(*) AS total_drivers
FROM
    police_log
WHERE
    driver_age IS NOT NULL
    AND driver_gender IS NOT NULL
    AND driver_race IS NOT NULL
    AND country_name IS NOT NULL
GROUP BY
    country_name, driver_gender, driver_race
ORDER BY
    country_name, total_drivers DESC;


""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+-----------------+---------------+---------------+-----------------+
| country_name   | driver_gender   | driver_race   |   average_age |   total_drivers |
+================+=================+===============+===============+=================+
| Canada         | F               | White         |          49   |            2246 |
+----------------+-----------------+---------------+---------------+-----------------+
| Canada         | M               | Black         |          48.8 |            2223 |
+----------------+-----------------+---------------+---------------+-----------------+
| Canada         | F               | Other         |          49.2 |            2221 |
+----------------+-----------------+---------------+---------------+-----------------+
| Canada         | F               | Asian         |          49.3 |            2220 |
+----------------+-----------------+---------------+---------------+-----------------+
| Canada         | M               | Hispan

# Query 6 Top 5 Violations with Highest Arrest Rates

In [443]:
mycursor.execute("""WITH violation_arrest_rate AS (SELECT violation,COUNT(*) AS total_stops,SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests, ROUND((SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS arrest_rate_percent FROM police_log WHERE violation IS NOT NULL GROUP BY violation)SELECT violation,total_stops,total_arrests,arrest_rate_percent FROM violation_arrest_rate ORDER BY arrest_rate_percent DESC LIMIT 5;
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+-------------+---------------+-----------------+-----------------------+
| violation   |   total_stops |   total_arrests |   arrest_rate_percent |
+=============+===============+=================+=======================+
| DUI         |         13075 |               0 |                     0 |
+-------------+---------------+-----------------+-----------------------+
| Signal      |         13112 |               0 |                     0 |
+-------------+---------------+-----------------+-----------------------+
| Seatbelt    |         13007 |               0 |                     0 |
+-------------+---------------+-----------------+-----------------------+
| violation   |             1 |               0 |                     0 |
+-------------+---------------+-----------------+-----------------------+
| Other       |         13194 |               0 |                     0 |
+-------------+---------------+-----------------+-----------------------+
